In [ ]:
#Citation for the below code
'''
Title: CCMEO LiCNN Prediction
Authors: Galen Richardson, Krishan Rajaratnam, Julie Lovitt
Date: 07-14-2021
Version: 1.1
https://www.nrcan.gc.ca/science-and-data/research-centres-and-labs/canada-centre-remote-sensing/21749
'''

In [1]:
from pathlib import Path
import numpy as np 
import os, shutil, time, csv,math
import tifffile as tif
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing import image as kimg
from tensorflow.keras import backend as K
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input,UpSampling2D, Conv2D 
from tensorflow.keras.layers import concatenate,Dropout, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from PIL import Image,ImageOps
import skimage.color as color
import cv2
import tifffile as TIF

#for dislaying results
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
plt.rcParams['figure.dpi'] = 150 
%matplotlib inline

In [2]:
#setting up project dir
project_dir = Path("").resolve().parents[0]

Model_Weights = str(project_dir / 'lichen_DRONE_333_VTAUG.h5')

input_image = str(project_dir / 'data/raw/d15_cor.tif')
drone_tiles = str(project_dir / 'data/raw/Drone_Tiles')
strips_folder= str(project_dir / 'data/raw/Drone_Strip')
drone_resize_img = str(project_dir / 'data/raw/d15_Drone_resize_rbg.png')

LiCNN_tiles = str(project_dir / 'data/raw/LICNN_Tiles')
LiCNN_Strips= str(project_dir / 'data/raw/LICNN_Strip')
LiCNN_img = str(project_dir / 'data/raw/d15_LICNN_img_frac_333_VTaug.png')

Transpose_img =  str(project_dir / 'data/raw/d15_overlay_LICNN_frac_333_Vtaug.png')

In [3]:

def breakup(input_image,drone_tiles):
    ts=240
    if os.path.exists(drone_tiles):
        shutil.rmtree(drone_tiles)
        os.makedirs(drone_tiles)
    else:
        os.makedirs(drone_tiles)
    img=tif.imread(input_image)
    img_shape = img.shape
    print (img_shape)
    x=math.ceil(img.shape[0]/ts)*ts-img.shape[0]
    y=math.ceil(img.shape[1]/ts)*ts-img.shape[1]
    if img.shape[0]/ts and img.shape[1]/ts != int():
        print ('lol this needs resizing')
        color = [0, 0, 0]
        img = cv2.copyMakeBorder(img, 0, x, 0, y, cv2.BORDER_CONSTANT, value=color)
        img_shape = img.shape
    tile_size = (ts, ts)
    offset = (ts,ts)
    global tile_list
    tile_list=[]
    folder_path=drone_tiles
    for i in range(int(math.ceil(img_shape[0]/(offset[1] * 1.0)))):
        for j in range(int(math.ceil(img_shape[1]/(offset[0] * 1.0)))):
            cropped_img = img[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
        # Debugging the tiles
            im_rgb = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
            tile=str(i) + ',' + str(j)
            img_path=folder_path + tile + ".png"
            tile_list.append(tile)
            cv2.imwrite(img_path, im_rgb)
    print ('Drone tiles created')
breakup(input_image,drone_tiles)

(22894, 25652, 3)
lol this needs resizing
Drone tiles created


In [4]:
def marry(tiles,strips,resize_img,modes):
    max_t=tile_list[-1]
    max_t=list(max_t.split(",")) 
    strip_list = []
    for i in range (int(max_t[0])+1):
        ilist=[]
        for j in range (int(max_t[1])+1):
            string=tiles+str(i)+','+str(j)+'.png'
            ilist.append(string)
            if int(j)==int(max_t[1]):
                imgs= [Image.open(h).convert("RGB")  for h in ilist ]
                imgs_comb = np.hstack( (np.asarray( h) for h in imgs ) )
                im_rgb = cv2.cvtColor(imgs_comb, cv2.COLOR_BGR2RGB)
                strip_list.append(strips +str(i)+'.png')
                cv2.imwrite((strips +str(i)+'.png'), im_rgb)
    print("creation of strips completed")
    imgs=[Image.open(i) for i in strip_list ]
    imgs_comb = np.vstack(np.asarray( v) for v in imgs )
    if modes==True:
        print('Applying median filter')
        imgs_comb = cv2.medianBlur(imgs_comb, 21) # Add median filter to image
        out_pil=Image.fromarray(imgs_comb)
        pixels = out_pil.load()
        for x in range(out_pil.size[0]): # for every pixel:
            for j in range(out_pil.size[1]):
                y=pixels[x,j]
                if y==(25,25,200): 
                    pixels[x,j]=(255,25,255)
        imgs_comb=np.array(out_pil)
    im_rgb = cv2.cvtColor(imgs_comb, cv2.COLOR_BGR2RGB)
    cv2.imwrite(resize_img, im_rgb)
    print("resized drone image create")

In [5]:
marry(drone_tiles,strips_folder,drone_resize_img,False)

C:\Users\gamr0\anaconda3\envs\RTX_LICNN\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if sys.path[0] == '':


creation of strips completed


C:\Users\gamr0\anaconda3\envs\RTX_LICNN\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


resized drone image create


In [6]:
def copy_make_out(drone_tiles, LiCNN_tiles, symlinks=False, ignore=None):
    pred_list =[]
    if os.path.exists(LiCNN_tiles):
        shutil.rmtree(LiCNN_tiles)
        os.makedirs(LiCNN_tiles)
    else:
        os.makedirs(LiCNN_tiles)
    for item in os.listdir(drone_tiles):
        s = os.path.join(drone_tiles, item)
        d = os.path.join(LiCNN_tiles, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)
    for path,subdir,files in os.walk(LiCNN_tiles):
        for file in files:
            pred_list.append(path+'\\'+file)
    print("Dir Copied and pred list created")
    return pred_list

In [7]:
#Defining the loss function
def dice_coef(y_true, y_pred, smooth=1e-6):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
    return dice
def ModifiedDiceLoss(y_true, y_pred, smooth=1e-6, modification = 1):
    cce = tf.losses.CategoricalCrossentropy()
    return 1 - dice_coef(y_true, y_pred) + modification * cce(y_true, y_pred)
def iou_coef(y_true, y_pred, smooth=1e-6):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def U_model():
    OUTPUT_CHANNELS = 3
    in1 = Input(shape=(None, None, 3 ))
    #changing all dropouts from .2 to .3
    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
    conv1 = Dropout(0.3)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = Dropout(0.3)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = Dropout(0.3)(conv3)
    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    conv4 = Dropout(0.3)(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)
    conv4 = Dropout(0.3)(conv4)
    
    up2 = concatenate([UpSampling2D((2, 2))(conv4), conv2], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv6 = Dropout(0.3)(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

    up2 = concatenate([UpSampling2D((2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv7 = Dropout(0.3)(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
    segmentation = Conv2D(3, (1, 1), activation='sigmoid', name='seg')(conv7)
    
    #defining inputs and outputs of model
    model = Model(inputs=[in1], outputs=[segmentation])
    #loss function needs to be SparseCategoricalCrossentropy
    '''Loss function Issues'''
    losses = {'seg': ModifiedDiceLoss}
    #losses = {'seg': DiceLoss}
    #tf.losses.CategoricalCrossentropy()
    metrics = {'seg': ['acc']}
    #compile the model to create it
    model.compile(optimizer="adam", loss = losses, metrics=['acc', dice_coef, iou_coef])
    
    #look into best loss function for this

    return model
model = U_model()
model.load_weights(Model_Weights)

In [8]:
def tf_predict(in_image):
    im=Image.open(in_image)
    im=np.array(im)
    im = np.expand_dims(im, axis=0)
    arg = tf.convert_to_tensor(im, dtype=tf.uint8)
    arg = tf.cast(arg, tf.float32) / 255.0
    with tf.device("/cpu:0"):
        pred_mask = model.predict(arg)
        pred_mask = tf.argmax(pred_mask, axis=-1)#which ever class the output data is most similar to, it gets assigned that
        pred_mask = tf.expand_dims(pred_mask, axis=-1)
    pred_mask= np.squeeze(pred_mask)
    pred_mask=pred_mask.astype(int)
    return pred_mask

In [9]:
def run_pred_make_out(drone_tiles, LiCNN_tiles):
    start1 = time.time()
    pred_list=copy_make_out(drone_tiles, LiCNN_tiles)
    for i in pred_list:
        open_i=cv2.imread(i)
        if np.max(open_i)<1:
            continue
        else:
            out=tf_predict(i)
            out_pil=Image.fromarray(out)
            out_pil = out_pil.convert('RGB')
            pixels = out_pil.load()
            for x in range(out_pil.size[0]): # for every pixel:
                for j in range(out_pil.size[1]):
                    y=pixels[x,j]
                        #pixels[x,j]=(200,200,25)cyan
                        #pixels[x,j]=(255,25,255)purple
                    if y==(2,2,2): 
                        pixels[x,j]=(25,25,255)#red
                    if y==(1,1,1): 
                        pixels[x,j]=(255,25,25)#blue
                    #if y==(0,0,0): 
                        #pixels[x,j]=(25,255,25)#purple
            out_pil = out_pil.convert('RGB')
            out_fin=np.array(out_pil)
            cv2.imwrite(i[:-3]+'png',out_fin)
    end1 = time.time()
    print("Predict Runtime: "+str("%.2f"%(end1 - start1)))
run_pred_make_out(drone_tiles, LiCNN_tiles)

Dir Copied and pred list created
Predict Runtime: 759.17


In [10]:
#create LICNN cover image
marry(LiCNN_tiles,LiCNN_Strips,LiCNN_img,False)

C:\Users\gamr0\anaconda3\envs\RTX_LICNN\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if sys.path[0] == '':


creation of strips completed


C:\Users\gamr0\anaconda3\envs\RTX_LICNN\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


resized drone image create


In [11]:
def Transpose_L_cover_on_drone(drone_resize_img,LiCNN_img,Transpose_img):
    start1 = time.time()
    Trans_pil = cv2.imread(LiCNN_img)
    OG_IMG= cv2.imread(drone_resize_img)
    print("Images read, now starting the merge")
    background = np.array(OG_IMG).astype(np.uint8)
    overlay = np.array(Trans_pil).astype(np.uint8)

    ignore_color=[0,0,0]
    ignore_color = np.asarray(ignore_color)
    mask = ~(overlay==ignore_color).all(-1)
    out = background.copy()
    out[mask] = background[mask] * 0.35 + overlay[mask] * 0.65

    cv2.imwrite(Transpose_img,out)
    end1 = time.time()
    print("Image Complete! Runtime: "+str("%.2f"%(end1 - start1)))
Transpose_L_cover_on_drone(drone_resize_img,LiCNN_img,Transpose_img)

Images read, now starting the merge
Image Complete! Runtime: 81.73
